In [8]:
import csv
import io
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [4]:
file_path = "monatszahlen2307_verkehrsunfaelle_10_07_23_nosum.csv"
encoding = 'utf-8'

with io.open(file_path, mode='r', encoding=encoding) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    #read column names
    col_names = next(reader)
    #read data
    data = np.array(list(reader))

# give each column name an id
print("Columns we are interested in:")
col_ids = {}
for i, col in enumerate(col_names[0:5]):
    col_ids[col] = i
    print(col_names[i], end=", ")
  
  # keep only the first five columns
data = data[:,0:5]

Columns we are interested in:
﻿"MONATSZAHL", AUSPRAEGUNG, JAHR, MONAT, WERT, 

In [5]:
print("First five rows of data:")
print(data[0:5,:])

First five rows of data:
[['Alkoholunfälle' 'insgesamt' '2023' '202301' '']
 ['Alkoholunfälle' 'insgesamt' '2023' '202302' '']
 ['Alkoholunfälle' 'insgesamt' '2023' '202303' '']
 ['Alkoholunfälle' 'insgesamt' '2023' '202304' '']
 ['Alkoholunfälle' 'insgesamt' '2023' '202305' '']]


In [6]:
# find unique entries
categories = np.unique(data[:,0])
types = np.unique(data[:,1])
print(categories)
print(types)

['Alkoholunfälle' 'Fluchtunfälle' 'Verkehrsunfälle']
['Verletzte und Getötete' 'insgesamt' 'mit Personenschäden']


In [9]:
# encode categorical data
le_1 = LabelEncoder()
le_2 = LabelEncoder()

data[:,0] = le_1.fit_transform(data[:,0])
data[:,1] = le_2.fit_transform(data[:,1])

categories_mapping = dict(zip(le_1.classes_, range(0, len(le_1.classes_))))
types_mapping = dict(zip(le_2.classes_, range(0, len(le_2.classes_))))

print("Encoded categories of accidents:",categories_mapping)
print("Encoded types of accidents:",types_mapping)

Encoded categories of accidents: {'Alkoholunfälle': 0, 'Fluchtunfälle': 1, 'Verkehrsunfälle': 2}
Encoded types of accidents: {'Verletzte und Getötete': 0, 'insgesamt': 1, 'mit Personenschäden': 2}
